# Fundamentals of Software Systems (FSS)
**Software Evolution – Part 02 Assignment**

## Submission Guidelines

To correctly complete this assignment you must:
* Carry out the assignment in a team of 2 to 4 students.
* Carry out the assignment with your team only. You are allowed to discuss solutions with other teams, but each team should come up its own personal solution. A strict plagiarism policy is going to be applied to all the artifacts submitted for evaluation.
* As your submission, upload the filled Jupyter Notebook (including outputs) together with the d3 visualization web pages (i.e. upload everything you downloaded including the filled Jupyter Notebook plus your `output.json`)
* The files must be uploaded to OLAT as a single ZIP (`.zip`) file by Dec 12, 2022 @ 23:55.


## Group Members
* Firstname, Lastname, Immatrikulation Number
* **TO BE FILLED**

## Task Context

In this assigment we will be analyzing the _elasticsearch_ project. You can use the code below to clone the repository and set up basic package dependencies. All following tasks should be done with the subset of commits from tag `v1.0.0` to tag `v1.1.0`.

In [51]:
from enum import Enum 

class Modification(Enum):
    ADDED = "Lines added"
    REMOVED = "Lines removed"
    TOTAL = "Lines added + lines removed"
    DIFF = "Lines added - lines removed"

## Task 1: Author analysis

In the following, please consider only `java` files.

The first task is to get an overview of the author ownership of the _elasticsearch_ project. In particular, we want to understand who are the main authors in the system between the two considered tags, the authors distribution among files and the files distribution among authors. To this aim, perform the following:
- create a dictionary (or a list of tuples) with the pairs author => number of modified files
- create a dictionary (or a list of tuples) with the pairs file => number of authors who modified the file
- visualize the distribution of authors among files: the visualization should have on the x axis the number of authors per file (from 1 to max), and on the y axis the number of files with the given number of authors (so for example the first bar represent the number of files with single author)
- visualize the distribution of files among authors: the visualization should have on the x axis the number of files per author (from 1 to max), and on the y axis the number of authors that own the given number of files (so for example the first bar represent the minor contributors, i.e., the number of authors who own 1 file)

Comment the two distribution visualizations.



Now, let's look at the following 3 packages in more details:
1. `src/main/java/org/elasticsearch/search`
2. `src/main/java/org/elasticsearch/index`
3. `src/main/java/org/elasticsearch/action`

Create a function that, given the path of a package and a modification type (see class Modification above), returns a dictionary of authors => number, where the number counts the total lines added or removed or added+removed or added-removed (depending on the given Modification parameter), for the given package. To compute the value at the package level, you should aggregate the data per file.

Using the function defined above, visualize the author contributions (lines added + lines removed). The visualization should have the author on the x axis, and the total lines on the y axis. Sort the visualization in decreasing amount of contributions, i.e., the main author should be the first.

Compare the visualization for the 3 packages and comment.

In [54]:
!pip install pydriller
!pip install bokeh
!pip install tqdm
!pip install pathlib
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install git

In [56]:
import pydriller
from collections import defaultdict
import tqdm
from pathlib import Path
from git import Repo

path_to_repo = "./mastodon"
repo_path = Path('./mastodon')

start_tag =  'v3.4.0'
current_tag = 'v3.5.3'
file_type = '.rb'


if repo_path.exists():
    print('Repository exists!')
else:
    # Clone the repository
    #  clone using the python git library
    Repo.clone_from('https://github.com/mastodon/mastodon', path_to_repo)

repo = pydriller.Repository(path_to_repo, from_tag=start_tag, to_tag=current_tag)

commits = list(repo.traverse_commits())
progress = tqdm.tqdm(unit="commit", total=len(commits))

files_by_author = defaultdict(set)
authors_per_file = defaultdict(set)
for commit in commits:
    for modified_file in commit.modified_files:
        author = commit.author.name.strip()
        file = str(modified_file.new_path)
        
        if not file:
            continue
            
        if not file.endswith(file_type):
            continue
        
        files_by_author[author].add(file)        
        authors_per_file[file].add(author)
    
    progress.update(1)


Repository exists!


100%|██████████| 1167/1167 [07:30<00:00,  2.59commit/s]


In [ ]:
files_by_author_count = { author : len(files_by_author[author]) for author in files_by_author}
files_by_author_count

authors_per_file_count = { file : len(authors_per_file[file]) for file in authors_per_file}
authors_per_file_count

{'lib/mastodon/version.rb': 2,
 'config/initializers/1_hosts.rb': 1,
 'app/lib/activitypub/activity.rb': 2,
 'app/services/activitypub/process_account_service.rb': 2,
 'app/services/fetch_link_card_service.rb': 3,
 'app/services/remove_status_service.rb': 2,
 'app/services/resolve_account_service.rb': 2,
 'app/workers/distribution_worker.rb': 2,
 'app/helpers/settings_helper.rb': 2,
 'config/application.rb': 4,
 'app/controllers/media_proxy_controller.rb': 2,
 'config/initializers/mail_delivery_job.rb': 1,
 'app/controllers/custom_css_controller.rb': 1,
 'app/controllers/activitypub/outboxes_controller.rb': 2,
 'lib/mastodon/search_cli.rb': 4,
 'app/models/account_stat.rb': 3,
 'app/models/concerns/account_counters.rb': 1,
 'db/post_migrate/20210526193025_remove_lock_version_from_account_stats.rb': 1,
 'db/schema.rb': 3,
 'lib/mastodon/maintenance_cli.rb': 2,
 'db/migrate/20190726175042_add_case_insensitive_index_to_tags.rb': 1,
 'db/migrate/20200620164023_add_fixed_lowercase_index_to_

In [ ]:
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output
from bokeh.models import ColumnDataSource, CategoricalColorMapper, PrintfTickFormatter, NumeralTickFormatter, Legend, CDSView, GroupFilter, CustomJS, BoxSelectTool, FactorRange
from bokeh.layouts import gridplot, column
from bokeh.models.widgets import Div
from bokeh.palettes import Spectral6, Pastel1, Category20c, Inferno256
import math
import os

output_notebook()

Loading BokehJS ...

In [ ]:
# visualize the distribution of authors among files: the visualization should have
# on the x axis the number of authors per file (from 1 to max), and on the y axis
# the number of files with the given number of authors (so for example the first
# bar represent the number of files with single author)

import numpy as np

# we only need a list where every item represents a file, and its value is the number of authors of that file
files_authors = list(authors_per_file_count.values())
max_authors = max(files_authors)

# we create an histogram with numpy, specifying the bins explicitly (instead of automatically) to have round values
hist_files_authors = np.histogram(files_authors, bins=range(1,max_authors + 2))
p = figure(toolbar_location=None, title="Distribution of authors among files", height=300, width=600)
p.vbar(x=hist_files_authors[1][:-1], top=hist_files_authors[0], line_color="white")

# # styling
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.x_range.start = 0.5
p.yaxis.minor_tick_line_color=None
p.xaxis.minor_tick_line_color=None
p.xaxis.ticker = list(range(1,9))
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

show(p)

In [ ]:
# visualize the distribution of files among authors: the visualization should have
# on the x axis the number of files per author (from 1 to max), and on the y axis
# the number of authors that own the given number of files (so for example the first
# bar represent the minor contributors, i.e., the number of authors who own 1 file)


# we only need a list where every item represents an author, and its value is the number of files modified by that author
authors_file = list(files_by_author_count.values())
max_files = max(authors_file)

hist_authors_file = np.histogram(authors_file, bins=10)
hist_authors_file

(array([28,  0,  0,  0,  0,  0,  0,  1,  0,  1]),
 array([  1. ,  48.2,  95.4, 142.6, 189.8, 237. , 284.2, 331.4, 378.6,
        425.8, 473. ]))

In [ ]:
bins = [round(x) for x in hist_authors_file[1]]

hist_authors_file = np.histogram(authors_file, bins=bins)
hist_authors_file

(array([28,  0,  0,  0,  0,  0,  0,  1,  0,  1]),
 array([  1,  48,  95, 143, 190, 237, 284, 331, 379, 426, 473]))

In [ ]:
bins_str = []
for index, el in enumerate(hist_authors_file[1][:-1]):
    bins_str.append(f"[{el},{hist_authors_file[1][index+1]})")
bins_str

['[1,48)',
 '[48,95)',
 '[95,143)',
 '[143,190)',
 '[190,237)',
 '[237,284)',
 '[284,331)',
 '[331,379)',
 '[379,426)',
 '[426,473)']

In [ ]:
p = figure(x_range=bins_str, toolbar_location=None, title="Distribution of files among authors", height=300, width=600)
p.vbar(x=bins_str, top=hist_authors_file[0], line_color="white")

# styling
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.yaxis.minor_tick_line_color=None
p.xaxis.minor_tick_line_color=None

show(p)

In [ ]:
class Modification(Enum):
    ADDED = "Lines added"
    REMOVED = "Lines removed"
    TOTAL = "Lines added + lines removed"
    DIFF = "Lines added - lines removed"
    
def calculate_author_contributions(repo, package, method):
    commits = list(repo.traverse_commits())
    progress = tqdm.tqdm(unit="commit", total=len(commits))
    
    result = defaultdict(int)
    for commit in commits:
        for modified_file in commit.modified_files:
            author = commit.author.name.strip()
            file = str(modified_file.new_path)
            
            if not file:
                continue
            
            # skip files outside the queried package
            if not package in file:
                continue
            
            if method == Modification.ADDED:
                result[author] += modified_file.added_lines
            elif method == Modification.REMOVED:
                result[author] += modified_file.deleted_lines
            elif method == Modification.TOTAL:
                result[author] += modified_file.added_lines + modified_file.deleted_lines
            elif method == Modification.DIFF:
                result[author] += modified_file.added_lines - modified_file.deleted_lines
        
        progress.update(1)

    return result

calculate_author_contributions(repo, "app/", Modification.ADDED)

100%|██████████| 1167/1167 [00:13<00:00, 84.70commit/s]


defaultdict(int,
            {'Zero King': 0,
             'Claire': 2607,
             'Eugen Rochko': 36493,
             'Jeong Arm': 81,
             'David Sterry': 40,
             'Takeshi Umeda': 111,
             'Holger': 5,
             'Daniel': 1,
             'matildepark': 5,
             'Truong Nguyen': 6,
             'Mashiro': 19,
             'Mélanie Chauvel': 32,
             'Sasha Sorokin': 30,
             'gunchleoc': 1,
             'mayaeh': 61,
             'Rens Groothuijsen': 8,
             'heguro': 1,
             'helloworldstack': 2,
             'dependabot[bot]': 1,
             'luzpaz': 5,
             'noiob': 1,
             'MitarashiDango': 1,
             'Josh Soref': 11,
             'chandrn7': 16,
             'Gomasy': 1,
             'CommanderRoot': 3,
             'rinsuki': 4,
             '0x2019': 8,
             'dogelover911': 1,
             'Chris Dzombak': 1,
             'Sara Golemon': 1,
             'Stefano Pigozzi': 6,

In [ ]:
method = Modification.TOTAL

# TODO

data_p1 = calculate_author_contributions(repo, "app/", method)
data_p2 = calculate_author_contributions(repo, "lib/", method)
data_p3 = calculate_author_contributions(repo, "public/", method)

100%|██████████| 1167/1167 [00:13<00:00, 87.23commit/s]


In [ ]:
data_p1

defaultdict(int,
            {'Zero King': 4,
             'Claire': 3744,
             'Eugen Rochko': 52400,
             'Jeong Arm': 103,
             'David Sterry': 50,
             'Takeshi Umeda': 184,
             'Holger': 9,
             'Daniel': 2,
             'matildepark': 6,
             'Truong Nguyen': 8,
             'Mashiro': 25,
             'Mélanie Chauvel': 45,
             'Sasha Sorokin': 42,
             'gunchleoc': 1,
             'mayaeh': 90,
             'Rens Groothuijsen': 12,
             'heguro': 2,
             'helloworldstack': 4,
             'dependabot[bot]': 2,
             'luzpaz': 10,
             'noiob': 2,
             'MitarashiDango': 2,
             'Josh Soref': 22,
             'chandrn7': 29,
             'Gomasy': 2,
             'CommanderRoot': 6,
             'rinsuki': 4,
             '0x2019': 12,
             'dogelover911': 2,
             'Chris Dzombak': 2,
             'Sara Golemon': 2,
             'Stefano Pigozzi'

In [ ]:
def plot_author_contribution(data, title):
    sorted_data = sorted(data.items(), key=lambda item: item[1], reverse=True)
    x_range = [item[0] for item in sorted_data]
    top = [item[1] for item in sorted_data]

    p = figure(x_range=x_range, 
               height=350, title=title,
               toolbar_location=None, tools="")
    p.vbar(x=x_range, top=top, width=0.9)

    # styling
    p.xgrid.grid_line_color = None
    p.xaxis.major_label_orientation = "vertical"
    p.y_range.start = 0
    p.yaxis.minor_tick_line_color=None
    p.xaxis.minor_tick_line_color=None
    # p.xaxis.ticker = list(range(1,9))
    p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

    show(p)

In [ ]:
plot_author_contribution(data_p1, "Author Contributions Search (Package 1)")
plot_author_contribution(data_p2, "Author Contributions Index (Package 2)")
plot_author_contribution(data_p3, "Author Contributions Action (Package 3)")


## Task 2: Knowledge loss

We now want to analyze the knowledge loss when the main contributor of the analyzed project would leave. For this we will use the circle packaging layout introduced in the "Code as a Crime Scene" book. This assignment includes the necessary `knowledge_loss.html` file as well as the `d3` folder for all dependencies. You task is to create the `output.json` file according to the specification below. This file can then be visualized with the files provided.

For showing the visualization, once you have the output as `output.json` you should
* make sure to have the `knowledge_loss.html` file in the same folder
* start a local HTTP server in the same folder (e.g. with python `python3 -m http.server`) to serve the html file (necessary for d3 to work)
* open the served `knowledge_loss.html` and look at the visualization

For the package you identify as the worst in terms of knowledge loss, investigate the author contributions using the function defined in the previous exercise and comment how the situation is, e.g. how big the gap between the main author and the second biggest contributor for the selected package is.

In [ ]:
repo = pydriller.Repository(path_to_repo, from_tag=start_tag, to_tag=current_tag)

method = Modification.TOTAL

commits = list(repo.traverse_commits())
progress = tqdm.tqdm(unit="commit", total=len(commits))

files_authors = defaultdict(dict)
for commit in commits:
    for modified_file in commit.modified_files:
        author = commit.author.name.strip()
        file = str(modified_file.new_path)

        if not file:
            continue

        if not file.endswith(file_type):
            continue

        if author not in files_authors[file]:
            files_authors[file][author] = 0
            
        if method == Modification.ADDED:
            files_authors[file][author] += modified_file.added_lines
        elif method == Modification.REMOVED:
            files_authors[file][author] += modified_file.deleted_lines
        elif method == Modification.TOTAL:
            files_authors[file][author] += modified_file.added_lines + modified_file.deleted_lines
        elif method == Modification.DIFF:
            files_authors[file][author] += modified_file.added_lines - modified_file.deleted_lines
        
    
    
    progress.update(1)

files_authors

 99%|█████████▉| 1158/1167 [00:13<00:00, 69.39commit/s]

defaultdict(dict,
            {'lib/mastodon/version.rb': {'Eugen Rochko': 24, 'Claire': 6},
             'config/initializers/1_hosts.rb': {'Claire': 2},
             'app/lib/activitypub/activity.rb': {'Claire': 4,
              'Eugen Rochko': 74},
             'app/services/activitypub/process_account_service.rb': {'Claire': 2,
              'Eugen Rochko': 62},
             'app/services/fetch_link_card_service.rb': {'Claire': 2,
              'Eugen Rochko': 103,
              'luzpaz': 2},
             'app/services/remove_status_service.rb': {'Claire': 8,
              'Eugen Rochko': 78},
             'app/services/resolve_account_service.rb': {'Claire': 7,
              'Eugen Rochko': 16},
             'app/workers/distribution_worker.rb': {'Claire': 2,
              'Eugen Rochko': 16},
             'app/helpers/settings_helper.rb': {'Eugen Rochko': 92,
              'gunchleoc': 1},
             'config/application.rb': {'Eugen Rochko': 19,
              'Claire': 23,
    

In [ ]:
files_authors

defaultdict(dict,
            {'lib/mastodon/version.rb': {'Eugen Rochko': 24, 'Claire': 6},
             'config/initializers/1_hosts.rb': {'Claire': 2},
             'app/lib/activitypub/activity.rb': {'Claire': 4,
              'Eugen Rochko': 74},
             'app/services/activitypub/process_account_service.rb': {'Claire': 2,
              'Eugen Rochko': 62},
             'app/services/fetch_link_card_service.rb': {'Claire': 2,
              'Eugen Rochko': 103,
              'luzpaz': 2},
             'app/services/remove_status_service.rb': {'Claire': 8,
              'Eugen Rochko': 78},
             'app/services/resolve_account_service.rb': {'Claire': 7,
              'Eugen Rochko': 16},
             'app/workers/distribution_worker.rb': {'Claire': 2,
              'Eugen Rochko': 16},
             'app/helpers/settings_helper.rb': {'Eugen Rochko': 92,
              'gunchleoc': 1},
             'config/application.rb': {'Eugen Rochko': 19,
              'Claire': 23,
    

In [ ]:
contribs_data = {}

for file, author_contribs in files_authors.items():
    total_contribs = sum(author_contribs.values())
    current_percentages = {}
    main_author = None
    max_author_percentage = 0
    if total_contribs != 0:
        for author, contribs in author_contribs.items():
            author_percentage = contribs / total_contribs
            if author_percentage > max_author_percentage:
                max_author_percentage = author_percentage
                main_author = author
        
        contribs_data[file] = {
            "main_author": main_author,
            "main_author_percentage": round(max_author_percentage, 2),
            "total_contribs": total_contribs
        }
        
        
contribs_data

{'lib/mastodon/version.rb': {'main_author': 'Eugen Rochko',
  'main_author_percentage': 0.8,
  'total_contribs': 30},
 'config/initializers/1_hosts.rb': {'main_author': 'Claire',
  'main_author_percentage': 1.0,
  'total_contribs': 2},
 'app/lib/activitypub/activity.rb': {'main_author': 'Eugen Rochko',
  'main_author_percentage': 0.95,
  'total_contribs': 78},
 'app/services/activitypub/process_account_service.rb': {'main_author': 'Eugen Rochko',
  'main_author_percentage': 0.97,
  'total_contribs': 64},
 'app/services/fetch_link_card_service.rb': {'main_author': 'Eugen Rochko',
  'main_author_percentage': 0.96,
  'total_contribs': 107},
 'app/services/remove_status_service.rb': {'main_author': 'Eugen Rochko',
  'main_author_percentage': 0.91,
  'total_contribs': 86},
 'app/services/resolve_account_service.rb': {'main_author': 'Eugen Rochko',
  'main_author_percentage': 0.7,
  'total_contribs': 23},
 'app/workers/distribution_worker.rb': {'main_author': 'Eugen Rochko',
  'main_author_p

## Output Format for Visualization

* `root` is always the root of the tree
* `size` should be the total number of lines of contribution
* `weight` can be set to the same as `size`
* `ownership` should be set to the percentage of contributions from the main author (e.g. 0.98 for 98% if contributions coming from the main author)

```
{
  "name": "root",
  "children": [
    {
      "name": "test",
      "children": [
        {
          "name": "benchmarking",
          "children": [
            {
              "author_color": "red",
              "size": "4005",
              "name": "t6726-patmat-analysis.scala",
              "weight": 1.0,
              "ownership": 0.9,
              "children": []
            },
            {
              "author_color": "red",
              "size": "55",
              "name": "TreeSetIterator.scala",
              "weight": 0.88,
              "ownership": 0.9,
              "children": []
            }
          ]
        }
      ]
    }
  ]
}
```

## JSON Export

For exporting the data to JSON you can use the following snippet:

```
import json

with open("output.json", "w") as file:
    json.dump(tree, file, indent=4)
```

In [ ]:
main_authors = defaultdict(int)
for info in contribs_data.values():
    main_authors[info['main_author']] += 1
    
sorted(main_authors.items(), key=lambda k: k[1], reverse=True)

[('Eugen Rochko', 434),
 ('Claire', 254),
 ('Takeshi Umeda', 10),
 ('Josh Soref', 8),
 ('luzpaz', 8),
 ('Jeong Arm', 3),
 ('chandrn7', 2),
 ('Holger', 2),
 ('dependabot[bot]', 2),
 ('0x2019', 2),
 ('Ikko Ashimine', 1),
 ('Truong Nguyen', 1),
 ('zunda', 1),
 ('Sumak', 1),
 ('MitarashiDango', 1),
 ('Rens Groothuijsen', 1),
 ('Yamagishi Kazutoshi', 1),
 ('Sara Golemon', 1)]

In [ ]:
main_author_by_contribs = defaultdict(int)
for file, author_contribs in files_authors.items():
    for author, contribs in author_contribs.items():
        main_author_by_contribs[author] += contribs

sorted(main_author_by_contribs.items(), key=lambda k: k[1], reverse=True)        

[('Eugen Rochko', 17168),
 ('Claire', 7192),
 ('Takeshi Umeda', 687),
 ('Jeong Arm', 133),
 ('Josh Soref', 132),
 ('chandrn7', 70),
 ('Holger', 30),
 ('luzpaz', 28),
 ('Yamagishi Kazutoshi', 20),
 ('0x2019', 13),
 ('dependabot[bot]', 10),
 ('zunda', 10),
 ('OSAMU SATO', 8),
 ('Truong Nguyen', 8),
 ('tkr', 6),
 ('Daniel', 4),
 ('Stefano Pigozzi', 4),
 ('helloworldstack', 4),
 ('rinsuki', 4),
 ('gunchleoc', 2),
 ('heguro', 2),
 ('Ikko Ashimine', 2),
 ('Sandro', 2),
 ('Wonderfall', 2),
 ('dogelover911', 2),
 ('Sumak', 2),
 ('MitarashiDango', 2),
 ('Rens Groothuijsen', 2),
 ('Sara Golemon', 2),
 ('David Sterry', 1)]

In [ ]:
main_author = "uboness"


def add_to_tree(branch, tree, info):
    
    if len(branch) == 1:
        if info["main_author"] == main_author:
            color = "red"
        else:
            color = "navy"
                        
        tree["children"].append({
            "author_color": color,
            "size": info["total_contribs"],
            "weight": info["total_contribs"],
            "name": branch[0],
            "ownership": info["main_author_percentage"],
            "children": []
        })
    else:
        node = branch[0]
        others = branch[1:]
        
        child = None
        for c in tree["children"]:
            if c["name"] == node:
                child=c
                break                
        
        if not child:
            child = {
                "name": node,
                "children": []
            }
            tree["children"].append(child)
            
        add_to_tree(others, child, info)
        
tree = {
    "name": "root",
    "children": []
}

for file, info in contribs_data.items():
    parts = file.split("/")
    add_to_tree(parts, tree, info)


tree
import json

with open("output-2022.json", "w") as file:
    json.dump(tree, file, indent=4)

In [ ]:
'src/java/main/Node.java'.split('/')

['src', 'java', 'main', 'Node.java']

In [ ]:
data_bucket = calculate_author_contributions(repo, "src/main/java/org/elasticsearch/search/aggregations/bucket", Modification.TOTAL)
plot_author_contribution(data_bucket, "Author contributions for search/aggregation/bucket")







































 34%|███▍      | 395/1167 [01:45<03:26,  3.74commit/s]





























100%|██████████| 1167/1167 [00:32<00:00, 69.39commit/s]































100%|██████████| 1167/1167 [00:14<00:00, 82.83commit/s]


## Task 3: Code Churn Analysis

The third and last task is to analyze the code churn of the _elasticsearch_ project. For this analysis we look at the code churn, meaning the daily change in the total number of lines of the project. Visualize the code churn over time bucketing the data by day. Remember that you'll need to interpolate the data for days when there are no commits. Chose an interpolation strategy and justify it.

Look at the churn trend over time and identify two outliers. For each of them:
- identify if it was caused by a single or multiple commits (since you are bucketing the data by day)
- find the hash of the involved commit(s)
- find the involved files
- look at the actual diff

Based on the above, discuss if the outlier is a false positive or should be a reason for concern.

In [ ]:
import datetime
import pytz
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output
from bokeh.models import ColumnDataSource, CategoricalColorMapper, PrintfTickFormatter, NumeralTickFormatter, Legend, CDSView, GroupFilter, CustomJS, BoxSelectTool, FactorRange
from bokeh.layouts import gridplot, column
from bokeh.models.widgets import Div
from bokeh.palettes import Spectral6, Pastel1, Category20c, Inferno256
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool, HoverTool
from bokeh.plotting import figure, show
from dateutil.rrule import rrule, DAILY


repo = pydriller.Repository(path_to_repo, from_tag=start_tag, to_tag=current_tag)

commits = list(repo.traverse_commits())
progress = tqdm.tqdm(unit="commit", total=len(commits))
timeseries_a = defaultdict(int)
timeseries_d = defaultdict(int)
timeseries_t = defaultdict(int)

for commit in commits:
    timeseries_a[commit.committer_date.date()] += commit.insertions
    timeseries_d[commit.committer_date.date()] += commit.deletions
    timeseries_t[commit.committer_date.date()] += commit.insertions + commit.deletions
    progress.update(1)
    

100%|██████████| 1167/1167 [00:53<00:00, 21.87commit/s]


In [ ]:
timeseries_a

defaultdict(int,
            {datetime.date(2021, 5, 16): 299,
             datetime.date(2021, 5, 17): 12,
             datetime.date(2021, 5, 18): 0,
             datetime.date(2021, 5, 19): 9,
             datetime.date(2021, 5, 20): 349,
             datetime.date(2021, 5, 22): 344,
             datetime.date(2021, 5, 24): 8,
             datetime.date(2021, 5, 25): 22,
             datetime.date(2021, 5, 27): 65,
             datetime.date(2021, 5, 29): 64,
             datetime.date(2021, 5, 30): 5,
             datetime.date(2021, 5, 31): 3,
             datetime.date(2021, 6, 1): 950,
             datetime.date(2021, 6, 2): 478,
             datetime.date(2021, 6, 3): 22,
             datetime.date(2021, 6, 7): 17,
             datetime.date(2021, 6, 10): 42,
             datetime.date(2021, 6, 14): 193,
             datetime.date(2021, 6, 15): 24,
             datetime.date(2021, 6, 18): 529,
             datetime.date(2021, 6, 21): 244,
             datetime.date(2021, 6, 23)

In [ ]:
def extrapolate_ts_day(timeseries, start_date, end_date):
    extrapolated_ts = []
    for dt in rrule(DAILY, dtstart=start_date, until=end_date):
        value = timeseries.get(dt.date(), 0)
        extrapolated_ts.append([dt, value])
    return extrapolated_ts


first_commit_date = commits[0].committer_date
last_commit_date = commits[-1].committer_date
full_timeseries_a = extrapolate_ts_day(timeseries_a, first_commit_date, last_commit_date)
full_timeseries_d = extrapolate_ts_day(timeseries_d, first_commit_date, last_commit_date)
full_timeseries_t = extrapolate_ts_day(timeseries_t, first_commit_date, last_commit_date)


In [ ]:
full_timeseries_a

[[datetime.datetime(2021, 5, 16, 23, 55, 7, tzinfo=<git.objects.util.tzoffset object at 0x7ff73011f130>),
  299],
 [datetime.datetime(2021, 5, 17, 23, 55, 7, tzinfo=<git.objects.util.tzoffset object at 0x7ff73011f130>),
  12],
 [datetime.datetime(2021, 5, 18, 23, 55, 7, tzinfo=<git.objects.util.tzoffset object at 0x7ff73011f130>),
  0],
 [datetime.datetime(2021, 5, 19, 23, 55, 7, tzinfo=<git.objects.util.tzoffset object at 0x7ff73011f130>),
  9],
 [datetime.datetime(2021, 5, 20, 23, 55, 7, tzinfo=<git.objects.util.tzoffset object at 0x7ff73011f130>),
  349],
 [datetime.datetime(2021, 5, 21, 23, 55, 7, tzinfo=<git.objects.util.tzoffset object at 0x7ff73011f130>),
  0],
 [datetime.datetime(2021, 5, 22, 23, 55, 7, tzinfo=<git.objects.util.tzoffset object at 0x7ff73011f130>),
  344],
 [datetime.datetime(2021, 5, 23, 23, 55, 7, tzinfo=<git.objects.util.tzoffset object at 0x7ff73011f130>),
  0],
 [datetime.datetime(2021, 5, 24, 23, 55, 7, tzinfo=<git.objects.util.tzoffset object at 0x7ff7301

In [ ]:
data = {
    "day": [x[0] for x in full_timeseries_a],
    "addition": [x[1] for x in full_timeseries_a],
    "deletion": [x[1] for x in full_timeseries_d],
    "total": [x[1] for x in full_timeseries_t]    
}

source = ColumnDataSource(data)

p = figure(height=300, width=980, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(data["day"][1], data["day"][10]))

hover = HoverTool(
    tooltips=[
        ("Day", "@day{%F}"),
        ("Lines added", "@addition{0,0}"),
        ("Lines removed", "@deletion{0,0}"),
        ("Lines total", "@total{0,0}")        
    ],
    formatters={
        '@day': 'datetime' # use 'datetime' formatter for '@date' field
    },

    mode='vline',
#     names=["addition"]
)

p.line('day', 'addition', source=source, name="addition", line_width=2)
p.line('day', 'deletion', source=source, line_color="red", line_width=2)
p.line('day', 'total', source=source, line_color="black", line_width=2)
p.add_tools(hover)
p.yaxis.axis_label = 'Churn'

select = figure(title="Zoom",
                height=130, width=980, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('day', 'addition', source=source)
select.line('day', 'deletion', source=source, line_color="red")
select.line('day', 'total', source=source, line_color="black")
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))


In [ ]:
spike1 = [c for c in commits if c.committer_date.date() == datetime.date(2022,2,26)]
[c.hash for c in spike1]

['3d60708508c6bfc5b6635aff0482d640a5f318ca',
 '6aef76b5cde2315135d53215d13a9b2ec0a1adaa',
 '48caeb9d659abd58ec7a9dc04f7365b35e314b74',
 '0dc57ab6ed67657e0a77e08bcd99c7b809fe5e42',
 '57814a98a9c8e4b106d44a31e36561f585f73bac']

In [ ]:
def print_diff(commits):
    additions = defaultdict(int)
    deletions = defaultdict(int)
    for commit in commits:
        for f in commit.modified_files:
            additions[str(f.new_path)] += f.added_lines
            deletions[str(f.new_path)] += f.deleted_lines

    all_files = set()
    all_files.update(additions.keys())
    all_files.update(deletions.keys())

    for f in sorted(all_files):
        print(f"+{additions.get(f,0)}\t -{deletions.get(f,0)} \t {f}")

In [ ]:
print_diff(spike1)

+2	 -0 	 app/controllers/admin/email_domain_blocks_controller.rb
+10	 -6 	 app/javascript/styles/mastodon/admin.scss
+3	 -37 	 app/lib/activitypub/activity/delete.rb
+8	 -3 	 app/lib/activitypub/activity/update.rb
+65	 -0 	 app/lib/activitypub/forwarder.rb
+16	 -3 	 app/views/admin/reports/show.html.haml
+2	 -1 	 app/workers/scheduler/email_domain_block_refresh_scheduler.rb
+1	 -0 	 config/locales/en.yml
+1	 -1 	 dist/nginx.conf


In [ ]:
spike2 = [c for c in commits if c.committer_date.date() == datetime.date(2022,2,11)]
[c.hash for c in spike2]

['aed98fd4ddc41b48522aeda609e0e30998f5e4e1',
 'f23c8dee2a43978e46310028907e382b7dd5c2cd',
 'ccf5bcc3dc884be88a10035305689f3857572599',
 '67034ea1ebc8c5d1df8f137fd67c8466aaf8d750',
 'f9f40069d5874948e7acf600179c2f602ee1e840',
 'b39127705714a732b0337b7a44211176cf2ad5a0',
 'ccad1a3caefe4f188e555d2bcab1f26f9b91bc4f',
 'c363fbe3a353f9f588af59e52aaf12a358e663bc',
 'cc7ae3932ca5542180d8a7200bc5c44cedaea976',
 '6f38765fccfeeed6c39a3b11962a518cae1cca4c',
 'd4e6774a0c88931d907f275821eb001e6dd2cb2d',
 'c9a52833b6840673bbed7454ca6b6b9cd88e7bfa',
 'a131f06e1299e21372f8f002c7959e54128be270',
 'a27729ee48aab4d75d562c2007b9967333c65d29',
 'd0fcf07436d158bcac2617d076a83d0aa49c39e6',
 '8f03b7a2fb4b420eb46942157160816185e81751',
 'ee47e2028bfc06ab9d35e5ea722073151e34042b',
 '4a0b6e3e5ea33fa1167f5820b442725bab16322f']

In [ ]:
print_diff(spike2)

+2	 -2 	 Gemfile.lock
+50	 -0 	 app/controllers/admin/reports/actions_controller.rb
+11	 -1 	 app/controllers/api/web/push_subscriptions_controller.rb
+11	 -2 	 app/javascript/mastodon/actions/notifications.js
+11	 -0 	 app/javascript/mastodon/features/notifications/components/column_settings.js
+35	 -0 	 app/javascript/mastodon/features/notifications/components/notification.js
+19	 -3 	 app/javascript/mastodon/locales/af.json
+53	 -37 	 app/javascript/mastodon/locales/ar.json
+32	 -16 	 app/javascript/mastodon/locales/ast.json
+19	 -3 	 app/javascript/mastodon/locales/bg.json
+19	 -3 	 app/javascript/mastodon/locales/bn.json
+132	 -116 	 app/javascript/mastodon/locales/br.json
+35	 -19 	 app/javascript/mastodon/locales/ca.json
+23	 -7 	 app/javascript/mastodon/locales/co.json
+29	 -13 	 app/javascript/mastodon/locales/cs.json
+19	 -3 	 app/javascript/mastodon/locales/cy.json
+157	 -141 	 app/javascript/mastodon/locales/da.json
+27	 -11 	 app/javascript/mastodon/locales/de.json
+93	 -2

In [ ]:
for commit in spike2:
    for f in commit.modified_files:
        if f.new_path is None:
            print(f.old_path)

In [ ]:
for commit in spike2:
    print(commit.msg)
    print("================================================================")

Bump sidekiq-unique-jobs from 7.1.12 to 7.1.15 (#17505)

Bumps [sidekiq-unique-jobs](https://github.com/mhenrixon/sidekiq-unique-jobs) from 7.1.12 to 7.1.15.
- [Release notes](https://github.com/mhenrixon/sidekiq-unique-jobs/releases)
- [Changelog](https://github.com/mhenrixon/sidekiq-unique-jobs/blob/main/CHANGELOG.md)
- [Commits](https://github.com/mhenrixon/sidekiq-unique-jobs/compare/v7.1.12...v7.1.15)

---
updated-dependencies:
- dependency-name: sidekiq-unique-jobs
  dependency-type: direct:production
  update-type: version-update:semver-patch
...

Signed-off-by: dependabot[bot] <support@github.com>

Co-authored-by: dependabot[bot] <49699333+dependabot[bot]@users.noreply.github.com>
Bump @babel/plugin-proposal-decorators from 7.16.7 to 7.17.0 (#17468)

Bumps [@babel/plugin-proposal-decorators](https://github.com/babel/babel/tree/HEAD/packages/babel-plugin-proposal-decorators) from 7.16.7 to 7.17.0.
- [Release notes](https://github.com/babel/babel/releases)
- [Changelog](https://g